In [10]:
%pip install -U spacy

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\K\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [11]:
%-m spacy download en_core_web_sm

UsageError: Line magic function `%-m` not found.


In [12]:
 %pip install networkx

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\K\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import networkx as nx 
import matplotlib.pyplot as plt
from tqdm import tqdm

import spacy
from spacy.matcher import matcher


nlp = spacy.load("en_core_web_sm")

In [4]:
pd.set_option('display.max_colwidth',200)

In [5]:
candidate_sentences = pd.read_csv("wiki_sentences_v2.csv")

candidate_sentences['sentence'].sample(5)

255         the latter film spawned a sequel: 28 weeks later .
1569                              kamal haasan debuted in 1960
1637             furious and heartbroken, sophie leaves akash.
3123         this brought wider success and acclaim to rampal.
789     : it's all about trying to hit the core and the heart.
Name: sentence, dtype: object

In [7]:
s = "I am a boy. He is Peter."
d = nlp(s)
for e in d.sents:
    print(e)

I am a boy.
He is Peter.


In [8]:
# count individual charaters

t = candidate_sentences['sentence'][0]
print(t,len(t))

confused and frustrated, connie decides to leave on her own. 60


In [9]:
doc = nlp(candidate_sentences['sentence'][0]) #count individual
print(doc,len(doc))

confused and frustrated, connie decides to leave on her own. 12


In [10]:
from spacy import displacy
doc = nlp("the drawdown process i s goveerned by astm standard d823")
displacy.render(doc, style="dep")

In [12]:
for tok in doc:
    print(tok.text, "-->", tok.dep_)

the --> det
drawdown --> compound
process --> ROOT
i --> nsubjpass
s --> aux
goveerned --> relcl
by --> agent
astm --> compound
standard --> pobj
d823 --> npadvmod


In [19]:
#task 1

def get_entities(sent):
    
  ent1 = ""           # Variable for storing the subject.
  ent2 = ""           # Variable for storing the object.

  prv_tok_dep = ""    # Variable for dependency tag of previous token in the sentence.
  prv_tok_text = ""   # Variable for previous token in the sentence.

  prefix = ""         # Variable for storing compounds.
  modifier = ""       # Variable for storing modifieres.

  # Loop through the tokens in the sentence.
  ## YOUR CODE GOES HERE.
    # Check if a token is a punctuation mark or not.
    ## YOUR CODE GOES HERE.
      # Check if a token is a compound one or not.
      ## YOUR CODE GOES HERE.
        # If yes, then store the token in the prefix variable.
        ## YOUR CODE GOES HERE.
        # Check if the previous token was also a compound one.
        ## YOUR CODE GOES HERE.
          # If yes, then update the prefix variable.
          ## YOUR CODE GOES HERE.
  for tok in nlp(sent):
    if tok.dep_ != "punct":
        if tok.dep_ == "compound":
            prefix = tok.text
            if prv_tok_dep == "compound":
                prefix = prv_tok_text + " "+ tok.text
        if tok.dep_.endswith("mod")== True:
            modifier = tok.text
            if prv_tok_dep == "compound":
                modifier = prv_tok_text + " "+ tok.text
        if tok.dep_.find("subj") == True:
            ent1 = modifier + " "+ prefix + " "+tok.text
            prefix = "" 
            modifier = ""
            prv_tok_dep = "" 
            prv_tok_text = ""
        if tok.dep_.find("obj") == True:
            ent2 = modifier + " "+ prefix + " "+tok.text
        prv_tok_dep = tok.dep_
        prv_tok_text = tok.text


          
      # Check if a token is a modifier or not.
      ## YOUR CODE GOES HERE.
        # If yes, then store the token in the modifier varible.
        ## YOUR CODE GOES HERE.
        # Check if the previous token was a compound one.
        ## YOUR CODE GOES HERE.
          # If yes, then update the modifier variable.
          ## YOUR CODE GOES HERE.
      
      # Check if a token is the subject.
      ## YOUR CODE GOES HERE.
        # If yes, then concatenate the modifier, prefix, and token
        # and assign the result to the subject variable (ent1).
        ## YOUR CODE GOES HERE.
        # Reset the following variables: prefix, modifier, prv_tok_dep, and prv_tok_text.
        ## YOUR CODE GOES HERE.    

      # Check if a token is the object.
      ## YOUR CODE GOES HERE.
        # If yes, then concatenate the modifier, prefix, and token 
        # and assign the result to the object variable (ent2).
        ## YOUR CODE GOES HERE.
         
      # Update the variable for the dependency tag for the previous token. 
      ## YOUR CODE GOES HERE.
      # Update the variable for the previous token in the sentence.
      ## YOUR CODE GOES HERE.

  return [ent1.strip(), ent2.strip()]

In [20]:
print(get_entities("the film had 200 patents"))

['film', '200  patents']


In [22]:
entity_pairs = []

for i in tqdm(candidate_sentences["sentence"]):
    entity_pairs.append(get_entities(i))

100%|██████████| 4318/4318 [00:15<00:00, 278.70it/s]


In [23]:
print(entity_pairs[0:2])

[['connie', 'own'], ['later  scream', 'distance']]


In [24]:
#task 2
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_sm")

matcher = Matcher(nlp.vocab)
pattern = [{"LIKE_EMAIL": True}]
matcher.add("EMAIL_ADDRESS",[pattern])

doc = nlp("This is an email adress: xxx@yyy.com")
matches = matcher(doc)

print(matches)
print(nlp.vocab[matches[0][0]].text)

[(16571425990740197027, 6, 7)]
EMAIL_ADDRESS


In [25]:
pattern = [{"POS": "PROPN"}]
matcher.add("PROPER_NOUN",[pattern])

doc = nlp("Cynthia Smith competed the task.")
matches = matcher(doc)

print(len(matches))
for match in matches:
    print( match, doc[match[1]:match[2]])
print(nlp.vocab[matches[0][0]].text)

2
(451313080118390996, 0, 1) Cynthia
(451313080118390996, 1, 2) Smith
PROPER_NOUN


In [26]:
pattern = [{"POS": "PROPN", "OP": "+"}]
matcher.add("PROPER_NOUN",[pattern])

doc = nlp("Cynthia Smith competed the task.")
matches = matcher(doc)

print(len(matches))
for match in matches:
    print( match, doc[match[1]:match[2]])
print(nlp.vocab[matches[0][0]].text)

3
(451313080118390996, 0, 1) Cynthia
(451313080118390996, 1, 2) Smith
(451313080118390996, 0, 2) Cynthia Smith
PROPER_NOUN


In [28]:
pattern = [{"POS": "PROPN", "OP": "+"},{"POS":"VERB"}]
matcher.add("PROPER_NOUN",[pattern], greedy="LONGEST")

doc = nlp("Cynthia Smith completed the task.")
matches = matcher(doc)
matches.sort(key= lambda x:x[1])
print(len(matches))
for match in matches:
    print( match, doc[match[1]:match[2]])
print(nlp.vocab[matches[0][0]].text)

1
(451313080118390996, 0, 3) Cynthia Smith completed
PROPER_NOUN


In [29]:
def get_predicate(sent):
    doc = nlp(sent)

    matcher = Matcher(nlp.vocab)

    pattern = [{'DEP':'ROOT'},{'DEP':'prep','OP':"?"},{'DEP':'agent','OP':"?"}]

    matcher.add("matching_1", [pattern])

    matches = matcher(doc)
    k = len(matches) - 1


    try: 
        span = doc[matches[k][1]:matches[k][2]]
        predicate = span.text
    except IndexError:
        predicate = ''
    return(predicate)

In [30]:
print(get_predicate("John completed the task"))

completed


In [31]:
print(get_predicate('confused and frustrated, Connie decides to leave on her own.'))

decides


In [32]:
candidate_sentences['sentence'][1]

'later, a woman’s scream is heard in the distance.'

In [33]:
predicates = [get_predicate(i) for i in tqdm(candidate_sentences['sentence'])]
print(predicates)

100%|██████████| 4318/4318 [00:16<00:00, 254.27it/s]

['decides', 'heard in', 'paralyzed by', 'set on', 'wails with', "'s", 'joined', 'revealed', 'revealed as', 'tried', 'went through', 'circling', 'supervised', 'features', 'is', 'injures', 'tasked with', 'attacked by', 'kota', 'return', 'stand', 'considered for', 'served as', 'completed on', 'released', 'released', 'distributed by', 'ended', 'chandler', 'dedicated to', 'approach', 'frees', 'disowns', 'appears as', 'credited as', 'stated', 'empathizing with', 'was', 'contributed to', 'took', 'changed', 'became', 'look like', 'perform', 'managed', 'released on', 'written by', 'conducted by', 'released', 'released', 'scheduled', 'offered in', 'includes', 'include', 'are', 'was', 'tend', 'include', 'prevalent in', 'is', 'are', 'had', 'supervises', 'oversee', 'credited', 'start in', 'started as', 'attended', 'use', 'take', 'equipped with', 'designed for', 'cooperates with', 'femis in', 'is', 'presented in', 'presented in', 'returns', 'breaks', 'imprisoned at', 'premiered in', 'doing', 'reshoo

In [34]:
print(pd.Series(predicates).value_counts()[:50])

is               401
was              334
are              103
released on       92
were              82
include           80
released          41
became            40
's                37
composed by       35
have              33
become            32
has               31
released in       28
been              27
included          26
received          26
had               26
made              24
be                22
considered        22
produced          21
called            20
used              18
directed by       17
written by        17
scheduled         17
stars             16
includes          15
went              14
hired             14
wrote             14
produced by       13
introduced in     13
wanted            12
won               12
sold              12
cast as           11
began in          11
began             11
set               11
gave              10
shot in           10
reported          10
features          10
going              9
need               9
produced in  